In [60]:
class CompatibleException(Exception):
    def __init__(self, message):
        self.message = message
        super().__init__(self.message)
    def __str__(self):
        return f"{self.message}"


class StrategyErrors:
    def __init__(self):
        self.message = None

    def UnCompatibleFilter(self, Strategy, Filter):
        self.message = f"Filter ({Filter.FilterIndex()}) not compatible with strategy ({Strategy.StrategyIndex()})"
        raise CompatibleException(self.message)


class StrategyHorizon:
    @classmethod
    def StrategyIndex(cls):
        return 'StrategyHorizon'

    def __init__(self):
        self._start = None
        self._end = None
        pass

    def input_params(self, start_horizon, end_horizon):
        self._start = start_horizon
        self._end = end_horizon

    def apply_filters(self, single_filter):
        if self.StrategyIndex not in single_filter.compatible:
            raise StrategyErrors().UnCompatibleFilter(Strategy=self, Filter=single_filter)


class CentralDotFilter:
    @classmethod
    def FilterIndex(cls):
        return 'CentralDotFilter'

    def __init__(self):
        self.compatible = [StrategyHorizon.StrategyIndex]



StrategyHorizon().apply_filters(CentralDotFilter())